In [1]:
import pandas as pd

# Consumable


In [4]:
import pandas as pd

# Load the Excel file
file_path = 'consumable.xlsx'  # Replace with the actual path to your file
df = pd.read_excel(file_path, sheet_name=0)

# Count the occurrences of each unique name in the "Stock name" column
result = df['Stock name'].value_counts().reset_index()
result.columns = ['Stock_name', 'Count']

# Merge the additional columns into the result
extra_columns = df[['Stock name', 'Price', 'Manufacturer', 'Catalog no.']].drop_duplicates()
result = result.merge(extra_columns, left_on='Stock_name', right_on='Stock name', how='left').drop(columns=['Stock name'])

# Calculate the "Total Price"
result['Total Price'] = result['Count'] * result['Price']

# Reorder columns
result = result[['Stock_name', 'Count', 'Manufacturer', 'Catalog no.', 'Price', 'Total Price']]

# Save the results to a new CSV file with the correct encoding
output_path = 'results_consumable.csv'
result.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')

print(f"Results have been saved to {output_path}")


Results have been saved to results.csv


C:\Users\Paul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


# Culture


In [7]:
import pandas as pd
import re
from collections import defaultdict

# Load the Excel file
file_path = 'culture.xlsx'  # Replace with the actual path to your file
df = pd.read_excel(file_path, sheet_name=0)

# Parse `Unit remarks` to extract amount and unit
def parse_unit_remarks(remarks):
    match = re.match(r'(\d+\.?\d*)\s*([a-zA-Zµ]+)', str(remarks))
    if match:
        amount = float(match.group(1))
        unit = match.group(2)
        return amount, unit
    return None, None

# Extract amount and unit from `Unit remarks`
df[['Amount_from_remarks', 'Unit_from_remarks']] = df['Units remarks'].apply(
    lambda x: pd.Series(parse_unit_remarks(x))
)

# Use `Size` and `Size unit` if `Unit remarks` is not available
df['Amount_final'] = df['Amount_from_remarks']
df['Unit_final'] = df['Unit_from_remarks']
df.loc[df['Amount_final'].isna(), 'Amount_final'] = df['Size']
df.loc[df['Unit_final'].isna(), 'Unit_final'] = df['Size unit']

# Group by `Stock name` and calculate totals for each unit
grouped = df.groupby('Stock name')
results = []

for stock_name, group in grouped:
    unit_totals = defaultdict(float)
    
    for _, row in group.iterrows():
        amount = row['Amount_final']
        unit = row['Unit_final']
        if pd.notnull(amount) and pd.notnull(unit):
            unit_totals[unit] += amount
    
    # Combine totals into a single string
    total_amounts = ', '.join(f"{value} {unit}" for unit, value in unit_totals.items())
    
    # Collect other data
    results.append({
        'Stock name': stock_name,
        'Count': len(group),
        'Manufacturer': group['Manufacturer'].iloc[0],
        'Catalog no.': group['Catalog no.'].iloc[0],
        'Price': group['Price'].iloc[0],
        'Total Amount & Unit': total_amounts,
        'Total Price': len(group) * group['Price'].iloc[0]  # Assuming the same price per unit
    })

# Convert results into a DataFrame
result_df = pd.DataFrame(results)

# Reorder columns
result_df = result_df[['Stock name', 'Count', 'Manufacturer', 'Catalog no.', 'Price', 'Total Amount & Unit', 'Total Price']]

# Save the results to a new CSV file with the correct encoding
output_path = 'results_culture.csv'
result_df.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')

print(f"Results have been saved to {output_path}")


Results have been saved to results_culture.csv
